# Heuristic Modelling

Here we first attempt to heuristically come up with probabilistic models of how humans take actions during blackjack. Then we use negative log-likelihoods to compare the heuristics and see which heuristic is mostly likely being followed by humans. We also compare these heuristic models with the optimal state-action function based model, and see which is more likely!


In [20]:
from game import *
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D  
import random
from matplotlib.pyplot import figure
from read_data import *
import math
from scipy.special import comb
from scipy.optimize import fminbound

In [21]:
# bootstrap code
def get_state(game):
    return (game.get_sum(game.get_player_hand()), game.get_hand_value(game.get_dealer_hand())[0], game.hasAce)

valid_states = [(x,y,z) for z in [True,False] for x in range(12,22) for y in range(2,12)]
states_without_ace = [(x,y,False) for x in range(12,22) for y in range(2,12)]
states_with_ace = [(x,y,True) for x in range(12,22) for y in range(2,12)]

def state_to_index(st):
    if st in valid_states:
        return valid_states.index(st)
    else:
        return -1

state_to_index((20, 2, True))
state_to_index((22, 2, True))

-1

### Model 1
#### P(win | state, action = hit_once)

This model suggests that humans base their decision (to hit or not) on the probability of winning (player sum being less than or equal to 21 and dealer sum being lesser than player sum) if they hit once and then stop. Basically we assume $$P(stand) = P(win | state, action = hit)$$  

We use monte-carlo sampling to calculate $P(stand) = P(win | state, action = hit)$. Optimally the player should base this decision on the possiblity of winning given he can hit multiple times, but we know that human mind doesn't always take decisions on a full-search exact-accuracy model. We also develop functions to compare our model with actual experiment data and measure the Maximum Likelihood Expectation.

In [22]:
def compute_log_likelihood(T, H, p):
    p = p if p > 0.0 else 0.0+1e-10
    p = p if p < 1.0 else 1.0-1e-10
    result = math.log(comb(T, H)) + (H*math.log(p) + (T-H)*math.log(1.0-p))
    return result

In [ ]:
SAMPLES = 500
def win_based_model(states, alpha):
    wins = np.zeros(len(states), dtype = float)
    for state in valid_states:
        for i in range(SAMPLES):
            ps, dc, ace = state
            game = BlackJack()
            game.set_state(ps, dc, ace)
            if game.hit() == Status.PLAYER_BUST:
                continue
            if game.stand() == Status.PLAYER_WON:
                wins[state_to_index(state)] += 1
    wins = wins/SAMPLES
    wins = (1-alpha)*wins + alpha*0.5
    return wins

def fit_win_based_model(alpha, human_data):
    probabilities = win_based_model(valid_states, alpha)
    total = 0
    for index,row in human_data.iterrows():
        total += compute_log_likelihood(row['Count'], row['HIT'], probabilities[state_to_index(row['State'])])
    return -1*total
    
data = get_human_results()
print("MLE for alpha = 1" + str(fit_win_based_model(0.0, data)))
minimizing_alpha = fminbound(fit_win_based_model, 0, 1, args=(data,))
print("The best fitted value of alpha" + str(minimizing_alpha))
fit_win_based_model(minimizing_alpha, data)

In [25]:
-1*compute_log_likelihood(20,15,0.75)

1.5978495582456187

### Model 2
#### P(win | state, action = stand)

This model suggests that humans base their decision (to stand or not) on the probability of winning if they stand in the current state. Basically we assume $$P(stand) = P(win | state, action = stand)$$ 

We use monte-carlo sampling to calculate $P(win | state, action = stand)$.

In [6]:
SAMPLES = 500
def win_based_model(states, alpha):
    wins = np.zeros(len(states), dtype = float)
    losses = np.zeros(len(states), dtype = float)
    for state in valid_states:
        for i in range(SAMPLES):
            ps, dc, ace = state
            game = BlackJack()
            game.set_state(ps, dc, ace)
            if game.stand() == Status.PLAYER_WON:
                wins[state_to_index(state)] += 1
    wins = wins/SAMPLES
    losses = 1 - wins
    losses = (1-alpha)*losses + alpha*0.5
    return losses

def fit_win_based_model(alpha, human_data):
    probabilities = win_based_model(valid_states, alpha)
    total = 0
    for index,row in human_data.iterrows():
        total += compute_log_likelihood(row['Count'], row['HIT'], probabilities[state_to_index(row['State'])])
    return -1*total
    
data = get_human_results()
print("MLE for alpha=0: " + str(fit_win_based_model(0.0, data)))
minimizing_alpha = fminbound(fit_win_based_model, 0, 1, args=(data,))
print("The best fitted value of alpha is " + str(minimizing_alpha))
fit_win_based_model(minimizing_alpha, data)

MLE for alpha=0 : 290.85705952508425
The best fitted value of alpha0.2478019710168759


228.9052224703976

### Model 3
#### Player Sum Based Model

Here, we make a decision based only on the player's sum. He stands at a sum of 17 or more, and hits otherwise.

In [16]:
def sum_based_model(states, alpha):
    prob_hit = np.zeros(len(states), dtype = float)
    for state in valid_states:
        ps, dc, ace = state
        if ps < 17:
            prob_hit[state_to_index(state)] = 1
        else:
            prob_hit[state_to_index(state)] = 0
    prob_hit = (1-alpha)*prob_hit + alpha*0.5
    return prob_hit


def fit_sum_based_model(alpha, human_data):
    probabilities = sum_based_model(valid_states, alpha)
    total = 0
    for index,row in human_data.iterrows():
        total += compute_log_likelihood(row['Count'], row['HIT'], probabilities[state_to_index(row['State'])])
    return -1*total
    
data = get_human_results()
print("MLE for alpha=0: " + str(fit_sum_based_model(0.1, data)))
minimizing_alpha = fminbound(fit_sum_based_model, 0, 1, args=(data,))
print("The best fitted value of alpha is " + str(minimizing_alpha))
fit_sum_based_model(minimizing_alpha, data)

MLE for alpha=0: 331.9128048013683
The best fitted value of alpha is 0.4860152273099868


210.48221244715316

### Model 4
#### Bust Based Policy

Here, we consider what is the probability of getting bust if we hit. Based on that we decided whether to hit or not. Basically, $$P(hit) = P(nobust|state,hit)$$

In [19]:
def bust_based_model(states, alpha):
    prob_not_bust = np.zeros(len(states), dtype = float)
    prob_hit = np.zeros(len(states), dtype = float)
    for state in valid_states:
        ps, dc, ace = state
        prob_not_bust[state_to_index(state)] = (21 - ps)/13
    prob_hit = (1-alpha)*prob_not_bust + alpha*0.5
    return prob_hit

def fit_bust_based_model(alpha, human_data):
    probabilities = bust_based_model(valid_states, alpha)
    total = 0
    for index,row in human_data.iterrows():
        total += compute_log_likelihood(row['Count'], row['HIT'], probabilities[state_to_index(row['State'])])
    return -1*total
    
data = get_human_results()
print("MLE for alpha=0: " + str(fit_bust_based_model(0.1, data)))
minimizing_alpha = fminbound(fit_bust_based_model, 0, 1, args=(data,))
print("The best fitted value of alpha is " + str(minimizing_alpha))
fit_bust_based_model(minimizing_alpha, data)

MLE for alpha=0: 348.1108206519913
The best fitted value of alpha is 0.570292789143172


261.8607410931742